In [1]:
import os
import cv2
import csv
import shutil
import extract_features as feature
from PIL import Image
import os

ModuleNotFoundError: No module named 'extract_features'

In [3]:
#########################################
###        Images preprocessing       ###
#########################################

def superpose_segmentation(normal_folder, segmentation_folder, output_folder):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Get a list of all the files in the normal folder
    normal_files = os.listdir(normal_folder)

    for file in normal_files:
        # Check if the file has a corresponding segmentation image
        segmentation_file = os.path.join(segmentation_folder, file.replace('.jpg', '_segmentation.png'))
        if os.path.exists(segmentation_file):
            try:
                # Open the normal image and the segmentation image
                normal_image = Image.open(os.path.join(normal_folder, file)).convert("RGBA")
                segmentation_image = Image.open(segmentation_file).convert("RGBA")

                # Resize the images to 256x256 pixels
                normal_image = normal_image.resize((256, 256))
                segmentation_image = segmentation_image.resize((256, 256))

                # Invert the segmentation image
                inverted_segmentation = Image.eval(segmentation_image, lambda x: 255 - x)

                # Create a binary mask from the inverted segmentation image
                mask = inverted_segmentation.split()[0].point(lambda x: 255 if x == 0 else 0).convert("L")

                # Apply the mask to the normal image
                normal_image.putalpha(mask)

                # Create a black background
                background = Image.new("RGBA", normal_image.size, (0, 0, 0, 255))
                # Composite the normal image with the black background
                result = Image.alpha_composite(background, normal_image)

                # Save the superposed image to the output folder
                output_file = os.path.join(output_folder, file)
                result.save(output_file, "PNG")

                print(f"Superposed {file}")
            except Exception as e:
                print(f"Error processing {file}: {str(e)}")
        else:
            print(f"No segmentation image found for {file}")

In [4]:
#########################################
###         Feature extraction        ###
#########################################

def extracting_features(image_path):
    """
    Extract features from an image.

    Args:
        image_path (str): Path to the input image.

    Returns:
        dict: Dictionary containing the extracted features.
    """
    image = cv2.imread(image_path)


    
    features = {}
    features['filename'] = os.path.basename(image_path)
    features['pigment_network_coverage'] = feature.measure_pigment_network(image)
    features['blue_veil_pixels'] = feature.measure_blue_veil(image)
    features['vascular_pixels'] = feature.measure_vascular(image)
    features['globules_count'] = feature.measure_globules(image)
    features['streaks_irregularity'] = feature.measure_streaks(image)
    features['irregular_pigmentation_coverage'] = feature.measure_irregular_pigmentation(image)
    features['regression_pixels'] = feature.measure_regression(image)

    return features

In [5]:
def save_features_to_csv(features_list, output_file):
    """
    Save a list of features dictionaries to a CSV file.

    Args:
        features_list (list): List of dictionaries containing image features.
        output_file (str): Path to the output CSV file.

    Returns:
        None
    """
    if not features_list:
        print("No features found.")
        return

    keys = features_list[0].keys()

    with open(output_file, 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=keys)
        writer.writeheader()
        writer.writerows(features_list)

    print(f"Features saved to {output_file}.")

In [6]:
def main():

    # Provide the paths to the folders containing the images
    normal_folder = r"/Users/niko/Documents/uni/6. semester/bsc/Project/eget/pad_segmentation_2/train/data/carcinoma_basocelular/"
    segmentation_folder = r"/Users/niko/Documents/uni/6. semester/bsc/Project/eget/pad_segmentation_2/train/mask/carcinoma_basocelular/"
    output_folder = r"/Users/niko/Documents/uni/6. semester/bsc/Project/eget/pad_segmentation_2/output/carcinoma_basocelular/"

    #Pre-process the images
    superpose_segmentation(normal_folder, segmentation_folder, output_folder)


    # Set the path for the output CSV file
    output_file = r"/Users/niko/Documents/uni/6. semester/bsc/Project/eget/pad_segmentation_2/output/carcinoma_basocelular_features.csv"

    # Initialize a list to store the extracted features
    features_list = []

    # Extracting features from all the pre-processed images
    for filename in [f for f in os.listdir(output_folder) if f.endswith('.jpg') or f.endswith('.png')]:
        # Extract features from the image
        image_path = os.path.join(output_folder, filename)
        features = extracting_features(image_path)

        # Append the features to the list
        features_list.append(features)

    # Save the features to a CSV file
    # if no file exists, create one
    if not os.path.exists(output_file):
        save_features_to_csv(features_list, output_file)
    else:
        os.remove(output_file)
        save_features_to_csv(features_list, output_file)

if __name__ == '__main__':
    main()

Superposed 2e95cce6-3684-4f1a-ade4-5fe09fb43d7d.jpg
Superposed 6441f5ca-e8d9-48af-aeb1-fdb242b50c5d.jpg
Superposed ac02d7b9-94b1-4975-883b-4c83a2a7e3d0.jpg
Superposed 6d9fe6e3-f1eb-4b66-9d63-6f5e6bceb3ea.jpg
Superposed 3024e0d0-bdb0-4b06-bc58-5e9b4be01cdd.jpg
Superposed 5db599a8-64d8-4bd6-834c-a6c0ca4b58b2.jpg
Superposed cadc757b-6aa7-469d-80f2-7556d8c247be.jpg
Superposed ac648630-cb0c-44ca-89c9-d955d2c7e6ce.jpg
Superposed 8ca79549-9fdf-459f-8378-0bbe2163207a.jpg
Superposed 40cfb53d-e078-481f-aa75-b4f5f5ae0ada.jpg
Superposed eed97191-51d3-459c-b552-844a73f57698.jpg
Superposed 419d41e8-a664-4d8f-851b-94a2d412d10a.jpg
Superposed 0c8373bb-cabc-4523-a3e4-19605bcbd95c.jpg
Superposed 05163334-ddd5-4d81-bcfc-d668244e715c.jpg
Superposed e774b6de-2915-44eb-a692-ac139086fd2c.jpg
Superposed a8589641-dff7-4562-abd3-dcda8bd62855.jpg
Superposed c642b8a2-b148-43a2-a3d9-bf9d8222c6db.jpg
Superposed 5d26dca4-2f58-4407-a835-4ded10258e56.jpg
Superposed 0134e4aa-1014-4523-bc6e-5ae724de74e3.jpg
Superposed e